In [25]:
import pyqubo
from pyqubo import Array, Binary, Constraint, Placeholder
import numpy as np
N, K = 3, 5
T = 7
z = 0.05
eps = 0.1

Sigma = np.random.rand(N, N) # UPDATE

Mu =  np.random.rand(N)


R = np.random.rand(T,N) # UPDATE

z_full = Array.create("z_full", shape= (N, K), vartype="BINARY")

r_eps_norm = Array.create("r_eps", shape=(K, 1), vartype="BINARY")

A_norm =  Array.create("A", shape=(K, 1), vartype="BINARY")
A_max = z

B_norm =  Array.create("B_full", shape=(T, K), vartype="BINARY")
B_max = np.random.rand(T) # UPDATE

C_norm =  Array.create("C", shape=(K, 1), vartype="BINARY")
C_max = eps*T


y = Array.create("y", shape=(T, 1), vartype="BINARY")

def binary2integer_norm(z):
    K = len(z)
    return sum([2**((k+1)-1-K)*z[k] for k in range(0,K)])


r_eps = z*(-1 + binary2integer_norm(r_eps_norm))
A = A_max*binary2integer_norm(A_norm)
C = C_max*binary2integer_norm(C_norm)

M = 10000

def IV(n):
    return (n*n - n)/2 + n

In [26]:
H_var = 0
for i in range(N):
    for j in range(N):
        H_var += binary2integer_norm(z_full[i])*binary2integer_norm(z_full[j])*Sigma[i,j]

In [27]:
H_exp = 0
for i in range(N):
        H_exp += Mu[i]*binary2integer_norm(z_full[i])

In [28]:
L1 = 100

H1 = L1*Constraint((-r_eps[0] - z + A[0])**2, "H1")

H1_dict = {'L1': 5.0}

In [29]:
L2s = ['L2_{}'.format(t) for t in range(T)]

H2 = sum([Placeholder(L2s[t])*Constraint((r_eps[0] 
                       - sum([R[t,j]*binary2integer_norm(z_full[j]) for j in range(N)]) 
                       - M*(1-y[t][0])+1-1 
                       + B_max[t]*binary2integer_norm(B_norm[t]))**2 , "H2_{}".format(t)) for t in range(T)])

M_dict = {"M": 1}
L2s_dict = dict(zip(L2s, [1]*T))
H2_dict = {**M_dict, **L2s_dict}

In [30]:
L3 = 100

H3 = L3*Constraint(((1-eps)*T - sum(y)[0] + C[0] )**2, "H3")

H3_dict = {'L3': 5.0}

In [31]:
L4 = 500

H4 = L4*Constraint((sum([binary2integer_norm(z_full[i]) for i in range(N)]))**2, "H4")

H4_dict = {'L4': 5.0}

In [32]:
H = H_var - H_exp + (H4+1-1) + (H1+1-1) + (H3+1-1) + (H2+1-1)
H_dict = {**H4_dict ,**H3_dict ,**H2_dict ,**H1_dict}

In [33]:
model = H.compile()

In [34]:
qubo = model.to_bqm(feed_dict= H_dict)

In [35]:
from dwave.samplers import SimulatedAnnealingSampler

In [36]:
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(qubo)
samples = model.decode_sampleset(sampleset)
#best_sample = min(samples, key=lambda s: s.energy)
#print(best_sample.sample)

#print(best_sample.constraints())

: 